In [ ]:
!pip install YahooFinancials
from yahoofinancials import YahooFinancials
import json

tech_stocks = ['AAPL']
#tech_stocks = ['AAPL', 'MSFT', 'INTC']
yahoo_financials_tech = YahooFinancials(tech_stocks)
weekly = yahoo_financials_tech.get_historical_price_data("2000-01-01", "2021-11-16", "daily")

import pandas as pd
with open('file.txt','w') as file:
     file.write(json.dumps(weekly))
import pandas as pd
df = pd.read_json('file.txt')

  Created wheel for YahooFinancials: filename=yahoofinancials-1.6-py3-none-any.whl size=15190 sha256=a4f75b331a008ad2533231bcefe055028aaed23d5585e559e25b6c361a92b5c2
  Stored in directory: /root/.cache/pip/wheels/4b/63/46/e7110bfee88685fe69e338d1b14d1748921862aa57b6705b60
Successfully built YahooFinancials


In [ ]:
adjclose, close, date, formatted_date, high, low, open, volume = [], [], [], [], [], [], [], []
for i in range(0,len(df['AAPL']['prices'])):
  adjclose.append(df['AAPL']['prices'][i]['adjclose'])
  close.append(df['AAPL']['prices'][i]['close'])
  date.append(df['AAPL']['prices'][i]['date'])
  formatted_date.append(df['AAPL']['prices'][i]['formatted_date'])
  high.append(df['AAPL']['prices'][i]['high'])
  low.append(df['AAPL']['prices'][i]['low'])
  open.append(df['AAPL']['prices'][i]['open'])
  volume.append(df['AAPL']['prices'][i]['volume'])
columns=['adjclose', 'close', 'date', 'formatted_date', 'high', 'low', 'open', 'volume']
dfData = pd.DataFrame(columns=columns)
dfData = pd.concat([pd.DataFrame([[adjclose[i], close[i], date[i], formatted_date[i], high[i], low[i], open[i], volume[i]]], columns = columns) for i in range(0,len(adjclose))], ignore_index=True)


In [ ]:
# k = dfData['close'].tolist()
# k.reverse()
print('as at: ' + str(dfData['formatted_date'].tolist()[-1]))

as at: 2021-11-15


In [ ]:
# univariate cnn lstm example
from numpy import array
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import TimeDistributed
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
import numpy as np
# split a univariate sequence into samples
def split_sequence(sequence, n_steps):
	X, y = list(), list()
	for i in range(len(sequence)):
		# find the end of this pattern
		end_ix = i + n_steps
		# check if we are beyond the sequence
		if end_ix > len(sequence)-1:
			break
		# gather input and output parts of the pattern
		seq_x, seq_y = sequence[i:end_ix], sequence[end_ix]
		X.append(seq_x)
		y.append(seq_y)
	return array(X), array(y)
 
# define input sequence
raw_seq = dfData['close']
# choose a number of time steps
n_steps = 4 # n_steps = n_seq*n_steps
print('step:', n_steps)
# split into samples
X, y = split_sequence(raw_seq, n_steps)
# reshape from [samples, timesteps] into [samples, subsequences, timesteps, features]
n_features = 1
n_seq = 2 # the number of subsequences
n_steps = 2 # the number of time steps per subsequence
X = X.reshape((X.shape[0], n_seq, n_steps, n_features))
# define model
model = Sequential()
model.add(TimeDistributed(Conv1D(filters=64, kernel_size=1, activation='relu'), input_shape=(None, n_steps, n_features)))
model.add(TimeDistributed(MaxPooling1D(pool_size=2)))
model.add(TimeDistributed(Flatten()))
model.add(LSTM(50, activation='relu'))
model.add(Dense(1))
model.compile(optimizer = 'adam', loss = 'mse', metrics=['mae', 'mape', 'acc'])
# fit model
model.fit(X, y, epochs=10, verbose=0)
# demonstrate prediction
k = dfData['close'].tolist()
k.reverse()
#The last 4 days closing price 
x_input = array([k[i] for i in range(n_steps*n_seq)])
x_input = x_input.reshape((1, n_seq, n_steps, n_features))
yhat = model.predict(x_input, verbose=0)
print(yhat)

step: 4
[[147.15219]]


In [ ]:
display(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
time_distributed (TimeDistri (None, None, 3, 64)       128       
_________________________________________________________________
time_distributed_1 (TimeDist (None, None, 1, 64)       0         
_________________________________________________________________
time_distributed_2 (TimeDist (None, None, 64)          0         
_________________________________________________________________
lstm (LSTM)                  (None, 50)                23000     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 23,179
Trainable params: 23,179
Non-trainable params: 0
_________________________________________________________________


None